In [1]:
from pprint import pprint
from torch_geometric.data import Data, DataListLoader, Dataset, InMemoryDataset, Batch
from torch_geometric.loader import DataListLoader, DataLoader
from torch_geometric.nn import *
from torch_geometric.utils import to_dense_adj, to_dense_batch, add_self_loops, remove_self_loops
from torch_geometric.nn.conv import MessagePassing
import torch
from torch import nn
import rdkit
from tqdm.auto import tqdm
import itertools
from rdkit import Chem
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
from rdkit import RDLogger

from copy import deepcopy
#from torch.utils.data import Dataset, DataLoader
from typing import Tuple, List, Dict, Union
from torch import Tensor
from torch_geometric.nn import MessagePassing, radius_graph
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import scatter
from torch_geometric.typing import (
    Adj,
    OptTensor,
    SparseTensor,
    pyg_lib,
    torch_sparse,
)
from torch_geometric.data import (
    Data,
    InMemoryDataset,
    download_url,
    extract_zip,
)
from rdkit import Chem
import os
# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')
cuda=torch.device('cuda') if torch.cuda.is_available() else 'cpu'
import sascorer
#torch.set_default_dtype(torch.float64)
from models import *
from rdkit.Chem.Crippen import MolLogP
from typing import List
from prolog import *
import torchlens as tl

In [2]:
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import negative_sampling
from torch_geometric.transforms import RandomLinkSplit
from sklearn.metrics import roc_auc_score

In [3]:
from torch_geometric.datasets import GNNBenchmarkDataset

In [4]:
ds=GNNBenchmarkDataset("datasets/gnnb-pattern",name='PATTERN')

Extracting datasets\gnnb-pattern\PATTERN\raw\PATTERN_v2.zip
Processing...
Done!


In [38]:
ds[0]

Data(x=[108, 3], edge_index=[2, 4884], y=[108])

In [13]:
class GCNLinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNLinkPredictor, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        # Two-layer GCN to compute node embeddings
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_index):
        # Dot product decoder: returns a score for each edge
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)

In [16]:
class TrainingContext:
    def __init__(self, cls, *args, **kwargs):
        self.inner = cls(*args, **kwargs).to(cuda)
        self.name = cls.__name__
        self.optim = torch.optim.AdamW(self.inner.parameters())
        self.sched = torch.optim.lr_scheduler.ExponentialLR(self.optim, gamma=0.98)
        self.num_parameters = sum(map(torch.numel,self.inner.parameters()))
        self.train_loss_record = dict()
        self.test_loss_record = dict()
        self.results = list()
        self.total_iters = 0
        self.running_loss = 0
        self.best_eval_loss = 999
        self.stopped = False
        self.batch_size=64
        self.train_loss_metric='MSE'
        self.eval_loss_metric='MAE'
        self.last_target_name='link prediction'
        self.last_dataset_name='GNN Benchmark: PATTERN'
        self.training=True
    def save(self, prefix='saves'):
        save_model(
            self.name,
            self.inner,
            optimizer=self.optim,
            scheduler=self.sched,
            loss_record={
                'train':self.train_loss_record,
                'test':self.test_loss_record,
            },
            total_training_iters=self.total_iters,
            last_batch_size=self.batch_size,
            loss_metric={
                'train':'MSE',
                'test':'MAE',
            },
            last_target_name=self.last_target_name,
            last_dataset_name=self.last_dataset_name
        )
    @classmethod
    def load(cls, name, class_, prefix='saves', training=False, override=None):
        if override is None:
            override=dict()
        checkpoint=torch.load(os.path.join(prefix,name,'checkpoint.pth'))
        self=cls(class_,**dict(**checkpoint['config'],**override))
        self.inner.load_state_dict(checkpoint['model_state_dict'])
        self.optim.load_state_dict(checkpoint['optimizer_state_dict'])
        self.sched.load_state_dict(checkpoint['scheduler_state_dict'])
        self.total_iters=checkpoint['total_training_iters']
        self.batch_size=checkpoint['last_batch_size']
        self.train_loss_record=checkpoint['loss_record']['train']
        self.test_loss_record=checkpoint['loss_record']['test']
        self.best_eval_loss=min(self.test_loss_record.values())
        self.training=training
        return self
    

In [18]:
split=RandomLinkSplit(num_val=0.1, num_test=0.2)

In [21]:
dl=DataLoader(ds,batch_size=64)

In [22]:
for batch in dl:
    break
split(batch)

(DataBatch(x=[7431, 3], edge_index=[2, 254783], y=[7431], batch=[7431], ptr=[65], edge_label=[509566], edge_label_index=[2, 509566]),
 DataBatch(x=[7431, 3], edge_index=[2, 254783], y=[7431], batch=[7431], ptr=[65], edge_label=[72794], edge_label_index=[2, 72794]),
 DataBatch(x=[7431, 3], edge_index=[2, 291180], y=[7431], batch=[7431], ptr=[65], edge_label=[145588], edge_label_index=[2, 145588]))

In [45]:
GNNBenchmarkDataset()[0], len(GNNBenchmarkDataset())

(Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34]), 1)

In [46]:
GNNBenchmarkDataset("datasets/gnnb-pattern",name='PATTERN')[0], len(GNNBenchmarkDataset("datasets/gnnb-pattern",name='PATTERN'))

(Data(x=[108, 3], edge_index=[2, 4884], y=[108]), 10000)

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score

# Load the KarateClub dataset
dataset = KarateClub()
data = dataset[0]

# If no node features are provided, use an identity matrix as one-hot encoding per node.
if data.x is None:
    data.x = torch.eye(data.num_nodes)

# Use RandomLinkSplit to create training, validation, and test splits.
# Note: is_undirected=True is set for an undirected graph.
transform = RandomLinkSplit(num_val=0.1, num_test=0.2, is_undirected=True)
train_data, val_data, test_data = transform(data)

class GCNLinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNLinkPredictor, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_index):
        # Dot product decoder to compute the edge score.
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNLinkPredictor(in_channels=train_data.x.size(1), hidden_channels=16, out_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

def train(model, opetimizer, data):
    model.train()
    optimizer.zero_grad()
    
    # Encode nodes using only the training graph (with positive edges).
    z = model.encode(data.x, data.edge_index)
    
    # Compute positive edge scores using training edges.
    pos_edge = data.edge_index
    pos_scores = model.decode(z, pos_edge)
    
    # Negative sampling on the training graph.
    neg_edge = negative_sampling(
        edge_index=pos_edge,
        num_nodes=data.num_nodes,
        num_neg_samples=pos_edge.size(1)
    )
    neg_scores = model.decode(z, neg_edge)
    
    # Binary cross entropy loss.
    pos_loss = -torch.log(torch.sigmoid(pos_scores) + 1e-15).mean()
    neg_loss = -torch.log(1 - torch.sigmoid(neg_scores) + 1e-15).mean()
    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test(model, train_data, val_data, test_data):
    model.eval()
    # Use the training graph to compute node embeddings.
    z = model.encode(train_data.x, train_data.edge_index).to(device)
    
    def compute_auc(split):
        # Evaluate using the provided edge_label_index and edge_label.
        scores = model.decode(z, split.edge_label_index)
        scores = scores.cpu()
        labels = split.edge_label.cpu()
        return roc_auc_score(labels, scores)
    
    auc_val = compute_auc(val_data)
    auc_test = compute_auc(test_data)
    return auc_val, auc_test

# Train the model
for epoch in range(1, 201):
    loss = train(model, optimizer, train_data)
    if epoch % 20 == 0:
        auc_val, auc_test = test(model, train_data, val_data, test_data)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {auc_val:.4f}, Test AUC: {auc_test:.4f}')


Epoch: 020, Loss: 1.0626, Val AUC: 0.7143, Test AUC: 0.6489
Epoch: 040, Loss: 0.9026, Val AUC: 0.6327, Test AUC: 0.7022
Epoch: 060, Loss: 0.8517, Val AUC: 0.5714, Test AUC: 0.6444
Epoch: 080, Loss: 0.8154, Val AUC: 0.5306, Test AUC: 0.6489
Epoch: 100, Loss: 0.8225, Val AUC: 0.4898, Test AUC: 0.6133
Epoch: 120, Loss: 0.7430, Val AUC: 0.4898, Test AUC: 0.6444
Epoch: 140, Loss: 0.8061, Val AUC: 0.4286, Test AUC: 0.6444
Epoch: 160, Loss: 0.8152, Val AUC: 0.4490, Test AUC: 0.6311
Epoch: 180, Loss: 0.7157, Val AUC: 0.4898, Test AUC: 0.6222
Epoch: 200, Loss: 0.8182, Val AUC: 0.3469, Test AUC: 0.5867


In [44]:
KarateClub()[0]

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [40]:
(test_data.x).argwhere()[:,0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
       device='cuda:0')

In [24]:
val_data.x

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')

Number of graphs in the dataset: 100
Example graph: Data(x=[108, 3], edge_index=[2, 4884], y=[108])
Epoch: 020, Loss: 1.2436, Val AUC: 0.5012, Test AUC: 0.5017
Epoch: 040, Loss: 1.1926, Val AUC: 0.5012, Test AUC: 0.5018
Epoch: 060, Loss: 1.1647, Val AUC: 0.5014, Test AUC: 0.5018
Epoch: 080, Loss: 1.1799, Val AUC: 0.5016, Test AUC: 0.5018
Epoch: 100, Loss: 1.1447, Val AUC: 0.5016, Test AUC: 0.5019
Epoch: 120, Loss: 1.1428, Val AUC: 0.5016, Test AUC: 0.5019


KeyboardInterrupt: 

In [10]:
batch

NameError: name 'batch' is not defined